# Python for Health - OB Ward Patient Flow DES Model

The following respresents a simple patient flow model for expectant mothers processes through Labor and Delivery in an in-hospital setting.

Such models can be incredibly helpful for modeling resource demand overtime, determining capacity and resource stretch, identifying process chokepoints and potential hazards.

Some specific Details:

- We will generate patient arrivals via a Poisson process
- Uses one Resource objects to model Observation (OBS) beds, Labor anr Deliver (LDR), and Post-Partum Recorvery (PP).
- Arrival rates and mean lengths of stay hard coded as constants. These statistics can be calculated based on hospital stats
- We also have encoded an arrival generator (initial delay and arrival stop time).

In [1]:
!pip install simpy

In [2]:
# Libraries to import
# NOTE: SimPy is not included in anaconda and will need to be installed
# before this notebook can be run
import simpy
import numpy as np
from numpy.random import RandomState

We will now assign a set of model parameters. Generally we will need to specify our overall model capacity as well as our resource utilization rate, where capacity is measured in terms of beds and utilization rate is determined in terms of hours

In [3]:
# We now need to assign model parameters
# This first set of parameters will define our utilization rate (in hours)
ARR_RATE = 0.4
MEAN_LOS_OBS = 3
MEAN_LOS_LDR = 12
MEAN_LOS_PP = 48

# Defining the size of our OB ward in terms of beds
CAPACITY_OBS = 2
CAPACITY_LDR = 6
CAPACITY_PP = 24

# Seed parameter for our randomizer
RNG_SEED = 6353

We will now create a series of helper functions that will create the rules for how our patients will migrate through the simulation.

The first is a patient generator model. This function will create the rules governing patient demand for services in our model.

Here we will use our predetermined parameters to model that demand by  generating patients according to a simple Poisson process

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        arr_rate : float
            exponential arrival rate
        initial_delay: float (default 0)
            time before arrival generation should begin
        stoptime: float (default Infinity)
            time after which no arrivals are generated
        prng : RandomState object
               Seeded RandomState object for generating pseudo-random numbers.

See [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.RandomState.html) for more details.



In [4]:
def patient_generator(env, arr_stream, arr_rate, initial_delay=0,
                      stoptime=simpy.core.Infinity, prng=RandomState(0)):
# ongoing count of patients created
    patients_created = 0

    # Yield for the initial delay
    yield env.timeout(initial_delay)

    # Generate arrivals as long as simulation time is before stoptime
    while env.now < stoptime:

        iat = prng.exponential(1.0 / arr_rate)

        # Sample los distributions
        los_obs = prng.exponential(MEAN_LOS_OBS)
        los_ldr = prng.exponential(MEAN_LOS_LDR)
        los_pp = prng.exponential(MEAN_LOS_PP)


        # Create new patient process instance
        patients_created += 1
        obp = obpatient_flow(env, 'Patient{}'.format(patients_created),
                             los_obs=los_obs, los_ldr=los_ldr, los_pp=los_pp)

        env.process(obp)

        # Compute next interarrival time

        yield env.timeout(iat)



Our next helper function has to establish the rules governing how a patient transits through the L&D process.

This Process function will model how a patient flows through system.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        name : str
            process instance id
        los_obs : float
            length of stay in OBS unit
        los_ldr : float
            length of stay in LDR unit
        los_pp : float
            length of stay in PP unit
The function will print status updates of a patient as she progresses through our process flow.

The function is designed to allow a patient to request, acquire, and then release a resource (bed) as she migrates through each step in our process.

Times to request are based on the patient generator function previosly generated.

Beds are allocated the moment they are available and requested. If there is a discrepency between bed requested and bed acquired, that time is noted as wait time.


In [5]:
def obpatient_flow(env, name, los_obs, los_ldr, los_pp):

    # Note the repetitive code and the use of separate request objects for each
    # stay in the different units.

    # OBS
    print("{} trying to get OBS at {:.4f}".format(name, env.now))
    bed_request_ts = env.now
    bed_request1 = obs_unit.request() # Request an OBS bed
    yield bed_request1
    print("{} entering OBS at {:.4f}".format(name, env.now))
    # this line is critical for calculating delays. If there is a
    # discrepancy between now (when the bed is released), and
    # when requested, we compute as a wait.
    if env.now > bed_request_ts:
        print("{} waited {:.4f} time units for OBS bed".format(name, env.now-  bed_request_ts))
    yield env.timeout(los_obs) # Stay in obs bed

    print("{} trying to get LDR at {:.4f}".format(name, env.now))
    bed_request_ts = env.now
    bed_request2 = ldr_unit.request()  # Request an LDR bed
    yield bed_request2

    # Got LDR bed, release OBS bed
    obs_unit.release(bed_request1)  # Release the OBS bed
    print("{} leaving OBS at {}".format(name, env.now))

    # LDR stay
    print("{} entering LDR at {:.4f}".format(name, env.now))
    if env.now > bed_request_ts:
        print("{} waited {:.4f} time units for LDR bed".format(name, env.now - bed_request_ts))
    yield env.timeout(los_ldr) # Stay in LDR bed

    print("{} trying to get PP at {:.4f}".format(name, env.now))
    bed_request_ts = env.now
    bed_request3 = pp_unit.request()  # Request a PP bed
    yield bed_request3

    # Got PP bed, release LDR bed
    ldr_unit.release(bed_request2)  # Release the obs bed
    print("{} leaving LDR at {:.4f}".format(name, env.now))

    # PP stay
    print("{} entering PP at {:.4f}".format(name, env.now))
    if env.now > bed_request_ts:
        print("{} waited {:.4f} time units for PP bed".format(name, env.now - bed_request_ts))
    yield env.timeout(los_pp) # Stay in PP bed
    pp_unit.release(bed_request3)  # Release the PP bed

    print("{} leaving PP and system at {:.4f}".format(name, env.now))



In [6]:
# Initialize a simulation environment
env = simpy.Environment()

prng = RandomState(RNG_SEED)


In [7]:
# Declare Resources to model all units
obs_unit = simpy.Resource(env, CAPACITY_OBS)
ldr_unit = simpy.Resource(env, CAPACITY_LDR)
pp_unit = simpy.Resource(env, CAPACITY_PP)

In [8]:
# Run the simulation for a while. Let's shut arrivals off after 50 time units.
runtime = 75
stop_arrivals = 50
env.process(patient_generator(env, "Type1", ARR_RATE, 0, stop_arrivals, prng))
env.run(until=runtime)

Patient1 trying to get OBS at 0.0000
Patient1 entering OBS at 0.0000
Patient1 trying to get LDR at 0.3475
Patient1 leaving OBS at 0.3474891089551544
Patient1 entering LDR at 0.3475
Patient2 trying to get OBS at 3.7668
Patient2 entering OBS at 3.7668
Patient3 trying to get OBS at 5.6439
Patient3 entering OBS at 5.6439
Patient1 trying to get PP at 6.1449
Patient1 leaving LDR at 6.1449
Patient1 entering PP at 6.1449
Patient4 trying to get OBS at 7.2625
Patient5 trying to get OBS at 8.9829
Patient3 trying to get LDR at 10.8140
Patient3 leaving OBS at 10.813985162144862
Patient3 entering LDR at 10.8140
Patient4 entering OBS at 10.8140
Patient4 waited 3.5515 time units for OBS bed
Patient4 trying to get LDR at 10.8690
Patient4 leaving OBS at 10.869006908518232
Patient4 entering LDR at 10.8690
Patient5 entering OBS at 10.8690
Patient5 waited 1.8861 time units for OBS bed
Patient4 trying to get PP at 11.4351
Patient4 leaving LDR at 11.4351
Patient4 entering PP at 11.4351
Patient2 trying to get

This represents a final runthrough of our patient flow system. As possible extensions we can begin to calculate metrics like average waittimes at each segment, min, max, and spread, average resource utilization etc.